In [1]:
import gcsfs

In [2]:
##gcs_browser = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='browser')

In [3]:
gcs_orig = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='cache')
gcs =  gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token=gcs_orig.session.credentials)

In [4]:
import os
import allel
import zarr

In [5]:
phase2path = os.path.join("ag1000g-release", "phase2.AR1", "variation", "main", "zarr", "biallelic", "ag1000g.phase2.ar1.pass.biallelic")

In [6]:
gcsmap = gcsfs.mapping.GCSMap(phase2path, gcs=gcs)

In [7]:
calldata= zarr.Group(gcsmap, read_only=True)

In [22]:
!wget --no-clobber ftp://ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/samples/samples.meta.txt

--2019-08-21 13:39:29--  ftp://ngs.sanger.ac.uk/production/ag1000g/phase2/AR1/samples/samples.meta.txt
           => ‘samples.meta.txt’
Resolving ngs.sanger.ac.uk (ngs.sanger.ac.uk)... 193.62.203.79
Connecting to ngs.sanger.ac.uk (ngs.sanger.ac.uk)|193.62.203.79|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /production/ag1000g/phase2/AR1/samples ... done.
==> SIZE samples.meta.txt ... 150452
==> PASV ... done.    ==> RETR samples.meta.txt ... done.
Length: 150452 (147K) (unauthoritative)

samples.meta.txt    100%[===================>] 146.93K   486KB/s    in 0.3s    

2019-08-21 13:39:31 (486 KB/s) - ‘samples.meta.txt’ saved [150452]



In [8]:
calldata["2L"]["calldata/GT"]

<zarr.core.Array '/2L/calldata/GT' (8906423, 1142, 2) int8 read-only>

In [9]:
calldata["2L"]["variants"]

<zarr.hierarchy.Group '/2L/variants' read-only>

In [10]:
g = calldata["2L"]["calldata/GT"]

In [11]:
g.chunks

(524288, 61, 2)

In [12]:
import pandas as pd

In [13]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")

In [14]:
metadata.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [15]:
metadata.population.unique()

array(['GHcol', 'GHgam', 'BFgam', 'BFcol', 'UGgam', 'GM', 'GW', 'KE',
       'CMgam', 'FRgam', 'GQgam', 'AOcol', 'GAgam', 'GNgam', 'GNcol',
       'CIcol'], dtype=object)

In [42]:
pop_selection = metadata.population.isin({'BFgam', 'BFcol', 'GHcol', 'AOcol'}).values
pop_select

array([ True,  True,  True, ..., False, False, False])

In [43]:
pop_subset = metadata[pop_selection]
pop_subset.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [44]:
geno = allel.GenotypeChunkedArray(g)
geno

<GenotypeChunkedArray shape=(8906423, 1142, 2) dtype=int8 chunks=(524288, 61, 2)
   nbytes=18.9G cbytes=-1 cratio=-20342270132.0
   compression=blosc compression_opts={'cname': 'zstd', 'clevel': 1, 'shuffle': -1, 'blocksize': 0}
   values=zarr.core.Array>

In [45]:
variants = calldata["2L"]["variants"]

In [46]:
pos_all = allel.VariantChunkedTable(variants, names=['POS'], index = 'POS')
pos_all

,POS
0,25050
1,51212
2,51214
...,...
8906420,49356425
8906421,49356426
8906422,49356429


In [47]:
pos_all = allel.SortedIndex(calldata["2L"]["variants/POS"])

In [48]:
pos_selection = pos_all.locate_range(2_000_000, 3_000_000)
pos_selection

slice(23444, 51125, None)

In [49]:
import numpy as np

In [50]:
mask = np.isin(pos_all, pos_selection)
mask

array([False, False, False, ..., False, False, False])

In [51]:
geno_subset = geno[pos_selection].subset(sel1=pop_selection)

In [52]:
geno_subset

<GenotypeArray shape=(27681, 300, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 1/1 1/1 0/1 0/0 ... 1/1 1/1 0/1 1/1 0/1
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

In [53]:
metadata.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [54]:
grp = metadata.groupby("population")

In [55]:
grp.indices

{'AOcol': array([880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892,
        893, 894, 895, 896, 897, 898, 899, 900, 901, 902, 903, 904, 905,
        906, 907, 908, 909, 910, 911, 912, 913, 914, 915, 916, 917, 918,
        919, 920, 921, 922, 923, 924, 925, 926, 927, 928, 929, 930, 931,
        932, 933, 934, 935, 936, 937, 938, 939, 940, 941, 942, 943, 944,
        945, 946, 947, 948, 949, 950, 951, 952, 953, 954, 955, 956, 957]),
 'BFcol': array([ 68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
         84,  85,  86,  87,  88,  89,  90,  94,  95,  96, 106, 107, 108,
        109, 110, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
        152, 153, 154, 162, 167, 168, 170, 171, 172, 175, 176, 177, 178,
        179, 180, 181, 183, 187, 190, 193, 195, 196, 198, 199, 200, 201,
        202, 208, 209, 212, 213, 216, 217, 225, 228, 231]),
 'BFgam': array([ 67,  81,  82,  83,  91,  92,  93,  97,  98,  99, 100, 101, 102,
        103, 104, 105, 111, 112, 11

In [56]:
grp.mean_coverage.agg(np.mean)

population
AOcol    29.872692
BFcol    33.142933
BFgam    34.180543
CIcol    30.688310
CMgam    32.030000
FRgam    31.204167
GAgam    30.554493
GHcol    31.917455
GHgam    31.268333
GM       30.398308
GNcol    30.885000
GNgam    28.610500
GQgam    36.198889
GW       31.059451
KE       36.606458
UGgam    32.185982
Name: mean_coverage, dtype: float64

In [59]:
subpops = {
    'all': list(range(len(pop_subset))),
    'BFcol': pop_subset[pop_subset.population == 'BFcol'].index.tolist(),
    'BFgam': pop_subset[pop_subset.population == 'BFgam'].index.tolist(),
    'GHcol': pop_subset[pop_subset.population == 'GHcol'].index.tolist(),
    'AOcol': pop_subset[pop_subset.population == 'AOcol'].index.tolist(),
}

In [60]:
geno_subset.count_alleles_subpops(subpops)

ValueError: index out of bounds

In [ ]:
allel.patterson_d()